# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
from google.colab import files

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
print("array", y.head().to_numpy())

array [0 0 0 1 0]


In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.loc[:, loans_df.columns != 'credit_ranking']

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
701,7.0,0.65,0.02,2.1,0.066,8.0,25.0,0.99720,3.47,0.67,9.5
56,10.2,0.42,0.57,3.4,0.070,4.0,10.0,0.99710,3.04,0.63,9.6
142,5.2,0.34,0.00,1.8,0.050,27.0,63.0,0.99160,3.68,0.79,14.0
863,7.2,0.62,0.06,2.5,0.078,17.0,84.0,0.99746,3.51,0.53,9.7
1387,7.4,0.64,0.07,1.8,0.100,8.0,23.0,0.99610,3.30,0.58,9.6


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)

# Fit the scaler to the features test dataset
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
num_features = X_train.shape[1]

# Review the number of features
print(f"{num_features}")

11


In [10]:
# Define the number of hidden nodes for the first hidden layer
num_hidden_nodes_1 = 6

# Define the number of hidden nodes for the second hidden layer
num_hidden_nodes_2 = 3

# Define the number of neurons in the output layer
num_output_neurons = 1

In [11]:
# Create the Sequential model instance
model = tf.keras.Sequential()

# Add the first hidden layer
model.add(Dense(num_hidden_nodes_1, activation='relu', input_shape=(num_features,)))

# Add the second hidden layer
model.add(Dense(num_hidden_nodes_2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(num_output_neurons, activation='sigmoid'))

In [12]:
# Display the Sequential model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [14]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=75, batch_size=32)

Epoch 1/75
40/40 [==============================] - 2s 6ms/step - loss: 0.8442 - accuracy: 0.4597
Epoch 2/75
40/40 [==============================] - 0s 8ms/step - loss: 0.7764 - accuracy: 0.4621
Epoch 3/75
40/40 [==============================] - 0s 6ms/step - loss: 0.7301 - accuracy: 0.4941
Epoch 4/75
40/40 [==============================] - 0s 3ms/step - loss: 0.6924 - accuracy: 0.5481
Epoch 5/75
40/40 [==============================] - 0s 6ms/step - loss: 0.6597 - accuracy: 0.6067
Epoch 6/75
40/40 [==============================] - 0s 10ms/step - loss: 0.6331 - accuracy: 0.6427
Epoch 7/75
40/40 [==============================] - 0s 6ms/step - loss: 0.6134 - accuracy: 0.6740
Epoch 8/75
40/40 [==============================] - 0s 3ms/step - loss: 0.5969 - accuracy: 0.6951
Epoch 9/75
40/40 [==============================] - 0s 7ms/step - loss: 0.5834 - accuracy: 0.7029
Epoch 10/75
40/40 [==============================] - 0s 4ms/step - loss: 0.5725 - accuracy: 0.7107
Epoch 11/75
40/40 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

# Display the model loss and accuracy results
print(f"Loss: {test_loss}")
print(f"Accuracy: {test_accuracy}")

Loss: 0.5267946124076843
Accuracy: 0.7281249761581421


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Export your model to a keras file
model.save(model_file_path)

In [17]:
# Download the keras file
files.download(model_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [18]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [19]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print("array", predictions[:5])

10/10 [==============================] - 0s 2ms/step
array [[0.5001971 ]
 [0.35865435]
 [0.7944177 ]
 [0.77485156]
 [0.96542645]]


In [20]:
# Save the predictions to a DataFrame and round the predictions to binary results
binary_predictions = np.round(predictions)
predictions_df = pd.DataFrame(binary_predictions, columns=['predictions'])
print(predictions_df.head())
print(predictions_df.tail())

   predictions
0          1.0
1          0.0
2          1.0
3          1.0
4          1.0
     predictions
315          1.0
316          0.0
317          0.0
318          1.0
319          1.0


### Step 4: Display a classification report with the y test data and predictions

In [21]:
# Print the classification report with the y test data and predictions
predictions_class = np.where(predictions >= 0.5, 1, 0)
print(classification_report(y_test, predictions_class))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73       154
           1       0.75      0.71      0.73       166

    accuracy                           0.73       320
   macro avg       0.73      0.73      0.73       320
weighted avg       0.73      0.73      0.73       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. The types of data that should be collected to build a recommendation system for student loan options include items such as payment history, location, STEM degree score, GPA ranking, alumni success, major of study, time to completion, finance knowledge and responsibility, cohort ranking, total loan score, financial aid score, and credit ranking.**

**The data should be collected because it makes it easier to predict if someone will repay a student loan. There are many factors that influence someone’s ability and priority when re-paying a student loan. It is best to look at all the factors since every situation is different on a case-by-case basis. When you look at the big picture, using AI and machine learning, you can predict whether a loan will probably be repaid, whether to approve or deny the loan request, and if approved, what interest rate would be appropriate.**

**For example, if a student has a very high GPA and great career success opportunities, but a history of overspending and debt, the Python code and results could look at them quite differently than someone who has a low GPA and low career success opportunities, but a solid history of full on-time payments and no debt. The former student may have the ability to repay a student loan, but not the financial responsibility since they may prioritize going on vacations before paying their loan. The latter student may have less of an ability to make payments and may struggle to make them, but they will likely make paying off their loan a priority over other unnecessary expenses in life, such as vacations.**


**2. Based on the data I chose to use in this recommendation system, my model would be using content-based filtering. When looking at certain attributes of a student’s life, we can correlate different pieces of content, such as loan history, credit history, GPA, etc. with other students who have similar histories or attributes, and use that to predict if, based on past-history and statistics, a student is likely going to pay off their loan.**


**3. There are some important real-world challenges that I would take into consideration while building a recommendation system for student loans.**

**First, collecting data on students, such as financial data, academic data, and PII (personally identifiable information) is very intrusive, and one must think about the ethics of data privacy. The data would need to be stored on state-of-the-art, high-quality servers that robustly prevent data breaches, theft from the dark web, and the selling of personal data to data brokerage entities.**

**Second, financial situations and life situations can change drastically over time. The data contained on each student may be out of date, or not reflect their current financial status or financial responsibility. For example, a student may have defaulted on a loan in the past due to being jobless and without a degree. Over the years, they could have inherited a windfall, earned a degree, advanced in their career, and started a family (with a spouse providing them a healthy dual income). For that reason, every loan seeker needs to be considered on a case-by-case basis, using updated data.**

